In [32]:
import torch
import torchvision
from torch.utils.data import DataLoader 
from torch import nn
from torch.nn import Conv2d,Linear,MaxPool2d,Flatten
from torch.utils.tensorboard import SummaryWriter

train_data = torchvision.datasets.CIFAR10("./CIFAR10",train=True,download = True,transform= torchvision.transforms.ToTensor())
test_data = torchvision.datasets.CIFAR10("./CIFAR10",train=False,download = True,transform= torchvision.transforms.ToTensor())
train_data_size= len(train_data)
test_data_size= len(test_data)
print("训练集的大小为:{}".format(train_data_size))
print("测试集的大小为:{}".format(test_data_size))
writer = SummaryWriter("logs")

Files already downloaded and verified
Files already downloaded and verified
训练集的大小为:50000
测试集的大小为:10000


In [33]:
train_dataloader=DataLoader(train_data,batch_size=64)
test_dataloader=DataLoader(test_data,batch_size=64)


In [34]:
class Miles(nn.Module):
    def __init__(self):
        super().__init__()
        # self.conv1= Conv2d(3,32,5,padding=2)
        # self.maxpool1=MaxPool2d(2)
        # self.conv2=Conv2d(32,32,5,padding=2)
        # self.maxpool2=MaxPool2d(2)
        # self.conv3=Conv2d(32,64,5,padding=2)
        # self.maxpool3=MaxPool2d(2)
        # self.flatten=Flatten()
        # self.linear1=Linear(1024,64)
        # self.linear2=Linear(64,10)
        self.model=nn.Sequential(
            Conv2d(3,32,5,padding=2),
            MaxPool2d(2),
            Conv2d(32,32,5,padding=2),
            MaxPool2d(2),
            Conv2d(32,64,5,padding=2),
            MaxPool2d(2),
            Flatten(),
            Linear(1024,64),
            Linear(64,10))
    def forward(self,x):
        # x=self.conv1(x)
        # x=self.maxpool1(x)
        # x=self.conv2(x)
        # x=self.maxpool2(x)
        # x=self.conv3(x)
        # x=self.maxpool3(x)
        # x=self.flatten(x)
        # x=self.linear1(x)
        # x=self.linear2(x)
        x=self.model(x)
        return x

miles = Miles()
loss_fn =  nn.CrossEntropyLoss()
learning_rate = 1e-2
optimizer = torch.optim.SGD(miles.parameters(),lr=learning_rate)
total_train_step = 0
total_test_step = 0 
epoch=10

In [35]:
for i in range (epoch):
    print("----------第{}轮训练开始----------".format(i+1))

    #train
    miles.train()
    for data in train_dataloader:
        imgs,targets = data
        outputs = miles(imgs)
        loss=loss_fn(outputs,targets)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_train_step+=1
        if total_train_step%100==0:
            print("训练次数:{},Loss:{}".format(total_train_step,loss.item()))
            writer.add_scalar("train_loss",loss.item(),total_train_step)

    #test
    miles.eval()
    total_accuracy = 0
    loss = 0
    with torch.no_grad():
        for data in test_dataloader:
            imgs,targets =data
            outputs = miles(imgs)
            loss+=loss_fn(outputs,targets)
            accuray=(outputs.argmax(1)==targets).sum()
            total_accuracy+=accuray
    total_test_step+=1
    print("整体测试集上的loss:{}".format(loss))
    print("整体测试集上的accuracy:{}".format(total_accuracy/test_data_size))
    writer.add_scalar("test_loss",loss.item(),total_test_step)
    writer.add_scalar("accuracy",total_accuracy/test_data_size,total_test_step)


----------第1轮训练开始----------
训练次数:100,Loss:2.2918455600738525
训练次数:200,Loss:2.281914710998535
训练次数:300,Loss:2.2822723388671875
训练次数:400,Loss:2.2420873641967773
训练次数:500,Loss:2.139047145843506
训练次数:600,Loss:2.0786049365997314
训练次数:700,Loss:2.0036063194274902
整体测试集上的loss:318.6272277832031
整体测试集上的accuracy:0.26829999685287476
----------第2轮训练开始----------
训练次数:800,Loss:1.9308443069458008
训练次数:900,Loss:1.899020791053772
训练次数:1000,Loss:1.987227201461792
训练次数:1100,Loss:2.007298469543457
训练次数:1200,Loss:1.7509864568710327
训练次数:1300,Loss:1.7258710861206055
训练次数:1400,Loss:1.784666657447815
训练次数:1500,Loss:1.8157312870025635
整体测试集上的loss:314.7364807128906
整体测试集上的accuracy:0.29319998621940613
----------第3轮训练开始----------
训练次数:1600,Loss:1.8074508905410767
训练次数:1700,Loss:1.663224697113037
训练次数:1800,Loss:1.9528231620788574
训练次数:1900,Loss:1.736617088317871
训练次数:2000,Loss:1.949045181274414
训练次数:2100,Loss:1.5611164569854736
训练次数:2200,Loss:1.5065138339996338
训练次数:2300,Loss:1.8286243677139282
整体测试集上的loss:271.2875